In [ ]:
# This file contains codes which downloads a file of known phishing URLs, extract phishing webpage prediction features 
# from each webpage of the collected phishing websites that collects sensitive data. The features are then stored in 
# the database. All functions here are imported from the files Functions and SQL_functions.

import csv
import mysql.connector
import mysql
from bs4 import BeautifulSoup
from Functions import *
from SQL_functions import *
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import sys
from datetime import datetime
import urllib3
from googleapiclient import discovery
import googleapiclient
import googleapiclient.discovery
import time
import json


def download_PhishTank_data():
    print('Starts downloading........')
    PhishTankURL = 'https://data.phishtank.com/data/online-valid.csv'
    file = 'C:/Users/tomna/OneDrive/Datasets/PhishTank/PhishTank.csv'
    urllib.request.urlretrieve(PhishTankURL,file)
    print('Download completed!')
    
    
def get_phishing_data(conn,cur):
    freeDomains = getFreeSubdomainList(cur)  # get a list of the most abused free domain names by attackers pre-loaded in the database
    TLDs = getTLDs(cur)  # get a list of known gTLDs and ccTLDs pre-loaded in the database
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}  # make a server to see a request coming from a browser and not a python engine. Some servers block request from non-browsers
    time_start = time.time()
    time1 = datetime.now().strftime('%Y-%m-%d %H:%M')
    print('--------Recording URLs starts at ' , time1,' ---------') 
    SN = 1
    # retrieve and open a file contain known phishing URLs. To each URL, download its webpage and extract the features
    file = 'C:/Users/tomna/OneDrive/Datasets/PhishTank/PhishTank.csv'
    with open(file, encoding='utf-8') as memoryFile:
        rowData = csv.reader(memoryFile, delimiter=',')       
        for row in rowData:
            recorded_URLs = []
            URLs = getURLsInDataset(cur)
            for url in URLs :
                recorded_URLs.append(url[1])
            URL = row[1]
            if URL in recorded_URLs :
                continue
            else :
                if len(URL) > 998 : # discard any URL above this long
                    continue
                else:
                    # web scraping each webpage to read its contents
                    try:
                        webpage = requests.get(URL,verify=False,headers=headers,timeout=120)   # verify=False means disabling SSL verification. Also set the timeout in sec to avoid long waits (see http://docs.python-requests.org/en/master/user/advanced/#timeouts)
                        webpage2 = webpage
                        try:
                            soup = BeautifulSoup(webpage.content,'html.parser')    
                        except html.parser.HTMLParseError as e:
                            continue
                        except UnicodeError as e:
                            continue              
                        # check all types of redirections associated with a webpage
                        url,URL,noRedirections,redirectType,shortURLFound  = detectAllURLRedirections(URL,webpage,cur,soup,headers)  # if there is redirection, url is the original url while URL is the final redirected url
                    except TypeError as e:
                        continue
                    except IndexError as e:   # issues with structuring of meta tag values
                        print('Error 0: ',e,'------',URL)
                        continue   
                    except requests.exceptions.RequestException as e:    # catch all types of exceptions related to connection and mistypying and skip to the next url (server is not reachable)
                        print('Error 1: ',e,'------',URL)
                        continue   
                    except urllib3.exceptions.LocationValueError as e:  # no host specified error
                        print('Error 2: ',e,'------',URL)  
                        continue                
                    if webpage2.status_code != 200:    # skip if the webpage is not found in the server (server is reachable)
                        continue      
                    # check if a search form exists in the webpage and drops down the webpage with one
                    seachFormExist = checkSearchForm(soup)
                    if seachFormExist == 'Yes' :   # filter out webpages with search forms  
                        continue
                    else:  
                        # check if a webpage is of a type html, htm, javascript, json or xml. Drops it if is not
                        try:
                            validPage,page_type = checkWebpageType(webpage2)     # Does the page's content in html, htm, javascript, json or xml format?
                        except KeyError as e:    # raise an error when ..........
                            print('Error 3: ',e,'------',URL)
                            continue
                        if validPage == 'False':
                            print('Error 4: ','------',URL)
                            continue
                        elif validPage == 'True':
                            # check if the page has a form & input tag
                            formChecker,inputChecker,countText,countEmail,countTel,countDate,countMonth,countPass,noPrompts,noWindowPrompts,noPopupWindow,noJSPrompts = checkFormInputTags(soup)                     
                            # decoding contents from their respective encoded formats to a readable format
                            try:                          
                                content = decodeWebpageContents(webpage2,soup)
                                content2 = content
                            except UnicodeDecodeError as e:
                                print('Error 5: ',e,'------',URL)
                                continue
                            # clean the contents by removing tags and white spaces
                            newContent = removeTags(content)
                            newContent = removeWhitespaces(newContent)    
                            htmlEncodedJS = 'No'
                            # check if the entire webpage is written in JavaScripts. If it is, return it in readable format
                            if formChecker == 'No' and inputChecker == 'No' and noJSPrompts == 0:  
                                 if 'document.write(unescape(' in content:    # if the whole webpage is encoded with JS
                                     decodedContent = decodeJavaScriptContents(soup)
                                     content2 = decodedContent
                                     soup = BeautifulSoup(decodedContent, 'html.parser')
                                     formChecker,inputChecker,countText,countEmail,countTel,countDate,countMonth,countPass,noPrompts,noWindowPrompts,noPopupWindow,noJSPrompts = checkFormInputTags(soup)
                                     htmlEncodedJS = 'Yes'
                                     decodedContent = removeTags(decodedContent)
                                     newContent = removeWhitespaces(decodedContent)
                            # tanslate the contents to English if they are not in English
                            translator = Translator()
                            try:
                                language = checkLanguage(soup,translator,newContent)
                            except AttributeError as e:    # if the webpage from the above function does not have a html or title tags
                                #print('Error 6: ',e,'------',URL)
                                language = 'en'
                                #continue
                                pass
                            except ValueError as e:    # error on translator.translate 
                                print('Error 7: ',e,'------',URL)
                                continue 
                            except requests.exceptions.ConnectionError as e:    # error on translator.translate 
                                print('Error 8: ',e,'------',URL)
                                continue 
                            if 'en' not in language and 'EN' not in language and language is not None and language != '':  
                                try:
                                    translatedText = translate(lugha,newContent,translator)
                                    newContent = translatedText
                                except ValueError as e:   # handle missed translation
                                    print('Error 8a: ',e,'------',URL)
                                    continue
                                except requests.exceptions.ConnectionError as e:   # handle missed translation
                                    print('Error 8b: ',e,'------',URL)
                                    continue
                                except AttributeError as e:   # handle missed translation
                                    #print('Error 8c: ',e,'------',URL)
                                    #continue
                                    pass
                            # check for the existence of login related phrases
                            allPhrases = getLoginPhrases(cur)
                            wordsCount,phraseMatchList = searchLoginWord(allPhrases,newContent)   # words are searched from a content without script texts
                            wordsCountScript = 0
                            phraseMatchListScript = []
                            if soup.findAll('script') and 'document.write(unescape(' not in str(webpage2.content):    # for webpages with script tags but not encoded
                                scriptTexts = JavaScriptText(soup)
                                wordsCountScript,phraseMatchListScript = searchLoginWord(allPhrases, scriptTexts)   # words are searched from javascript texts   
                            # start extracting features from the webpages meeting the conditions
                            if (formChecker == 'Yes' and inputChecker == 'Yes'  and (wordsCount >= 0 or wordsCountScript >= 0) and ((countText > 1 or countEmail != 0  or countTel != 0  or countDate != 0  or countMonth != 0 or countPass != 0) or (noJSPrompts != 0 and (wordsCount != 0 or wordsCountScript != 0)))) :
                            FQDN,domainIdentity,mainDomain,URL_path,freeDomainUsed,FREEDomain,IPAddressUsed,IPAddress,PortNoUsed,PortNo,ccTLDUsed,ccTLD,gTLDUsed = getDomainIdentity(freeDomains,URL,TLDs)
                            IPAddressUsed = check_IP4_in_hostname(hostname) 
                            # extract the feature related to IP address
                            if IPAddressUsed == 'Yes':  
                                try:
                                    hostname = reverseIPLookup(IPAddress) # This will be the host/sub domain of the web page
                                except socket.gaierror as e:   # socket connection error
                                    print('Error 10: ',e,'------',URL)
                                    continue
                                except UnicodeError as e:   
                                    print('Error 11: ',e,'------',URL)
                                    continue
                                except OSError as e:   
                                    print('Error 11b: ',e,'------',URL)
                                    continue
                                if hostname != '':
                                    URLPrefix = getURLPrefix(URL)
                                    URL = URLPrefix + ':'  + '//' + hostname + '/' + URL_path
                                    FQDN,domainIdentity,mainDomain,URL_path,freeDomainUsed,FREEDomain,IPAddressUsed,IPAddress,PortNoUsed,PortNo,ccTLDUsed,ccTLD,gTLDUsed = getDomainIdentity(freeDomains,URL,TLDs)
                                else:  
                                    print('Hostname not returned! --- ', URL)
                                    continue
                            # extract the feature related to void and same webpage hyperlink ratio
                            try:
                                dominantDomain,totalLinks,voidLinks,samePageLinks,hyperlinkList = getHyperlinkDominantDomain(soup,freeDomains,TLDs,mainDomain)
                            except IndexError as e: # in strange few webpages, all links may not to be extracted (e.g https://www.bible.com/en-GB/sign-in). We need to skip such webpages
                                print('No hyperlinks found --- ', URL)
                                continue
                            if mainDomain == dominantDomain: 
                                hyperlinkDomainMatched = 'Yes'
                            else:
                                hyperlinkDomainMatched = 'No'  
                            if totalLinks != 0 :  
                                linkRatio = (voidLinks + samePageLinks)/totalLinks
                                linkRatio = float(str(round(linkRatio,2)))
                                otherLinks = totalLinks - (voidLinks + samePageLinks)
                            else:
                                linkRatio = -1
                                otherLinks = -1                
                            hostIPList = getIPAdresses(FQDN) # we avoid collecting URLs which can not be resolved as IPs are part of the features. Subdomains may be hosted in separate servers therefore we need to query using FQDN and not mainDomain    
                            if not hostIPList: # IPs list is empty
                                print('No returned IP addresses --- ', URL)
                                continue
                            # extract features related to URL encoded characters
                            else:  # IPs of a domain were returned
                                redirectionCharacter = checkRedirectionCharacter(URL)  
                                domainEncoded,pathEncodedCharacters = checkURLEncoding(FQDN,URL_path) 
                                #extract feature related to URL's port number
                                URLPrefix = getURLPrefix(URL)                                                                
                                if PortNoUsed == 'Yes': 
                                    standardPort = determineStandardPort(URLPrefix,PortNo)
                                else:
                                    standardPort = 'No'  
                                # extract features related to domain identity, and canonical and alternative hyperlinks
                                domainIdentityCounts = domainIdentityInWebapge(content2,domainIdentity)   # count number of appearances of a domain identity in the content of a webpage with tags  
                                domainInCopyright = domainIdentityInCopyright(soup,domainIdentity)  # check if the domain identity exists in a copyright info 
                                try:
                                    domainInCanonical = canonicalURL(URL,soup,mainDomain,freeDomains,TLDs)  # check if the webpage's URL and canonical URL share the same domain name 
                                    domainInAlternate = alternateURL(URL,soup,mainDomain,freeDomains,TLDs)  # check if the webpage's URL and alternate URL share the same domain name                            
                                except KeyError as e:
                                    print('Error in canonical or alternate URL! --- ', URL)
                                    continue
                                # extract features related to domain positioning, number of dots in the URL, use of obfuscation characters in the URL and URL length
                                outPosition = domainOutPosition(URL,FQDN,URL_path,TLDs)  
                                noDotsFQDN,noDotsURLPath = numberOfDots(FQDN,URL_path) 
                                obfuscFQDN,obfuscPath = URLObfuscationCharacters(FQDN,URL_path) 
                                lengthFQDN,lengthPath,noSlashes = checkURLLength(URL,FQDN,URL_path) 
                                # extract features related to domain validity and domain age
                                try:
                                    domainValidity,domainAge = domainWHOISrecords(mainDomain) 
                                except ConnectionRefusedError as e:
                                    print('Connection Refused Error --- ', URL)
                                    continue
                                except ConnectionResetError as e:
                                    print('Connection Reset Error --- ', URL)
                                    continue
                                except OSError as e:
                                    print('OS Error --- ', URL)
                                    continue
                                formHandlerStatus,FHDomainList = foreignFormHandler(soup,mainDomain,dominantDomain,freeDomains,TLDs)  # One login webpage may have more than one form submissions
                                if FHDomainList:   
                                    noOfFHdomains = len(FHDomainList)
                                    if noOfFHdomains == 1:
                                        if FHDomainList[0] != mainDomain:
                                            FHDomainValidity,FHDomainAge = domainWHOISrecords(FHDomainList[0]) 
                                        else:
                                            FHDomainValidity = domainValidity
                                            FHDomainAge = domainAge
                                    elif noOfFHdomains > 1:
                                        FHDomainValidityList = []
                                        ages = []
                                        for FHDomain in FHDomainList:
                                            try:
                                                FHDomainValidity,FHDomainAge = domainWHOISrecords(FHDomain)
                                            except ConnectionResetError as e: 
                                                continue
                                            except ConnectionAbortedError as e: 
                                                print('ConnectionAbortedError ------ ',URL)
                                                continue
                                            if FHDomainAge != -1 :
                                                FHDomainValidityList.append(FHDomainValidity)
                                                ages.append(FHDomainAge)
                                            else:
                                                continue
                                        try:
                                            FHDomainValidity = mostCommonItem(FHDomainValidityList)
                                            FHDomainAge = min(ages)  # find the least age
                                        except IndexError as e:   # if one of the above lists are empty, meaning no FHs were found with whois records (though they might be several FHs)
                                            print('Error 12: ',e,'----',URL,'-----',FHDomain,'-----',FHDomainAge)  
                                            FHDomainValidity = 'Unknown'
                                            FHDomainAge = -1
                                            pass
                                else:  # if no FHs were found. This is possible if the webpage uses javascript to send data and not a html form
                                    FHDomainValidity = 'Unknown'
                                    FHDomainAge = -1         
                                # extract data related to SSL certificates
                                try:                        
                                    certificateExist,OID,organizationName,countryName,jurisdictionCountryName,certificateVerificationErrorType = verifySSLCertificate(URL,FQDN,headers)   # FEATURES 23  
                                except TimeoutError as e: # error in connecting to the server to load the certificate
                                    print('Error 13:a ',e,'------',URL)
                                    continue
                                try:
                                    certificateType = checkSSLCertificateType(certificateExist,OID,organizationName,jurisdictionCountryName,certificateVerificationErrorType)
                                except UnboundLocalError as e: 
                                    print('Error 13b: ',e,'------',URL)
                                    continue
                                try:
                                    countryMatch = verifyRegisteredCountry(hostIPList,ccTLDUsed,ccTLD,gTLDUsed,certificateType,jurisdictionCountryName,countryName) # FEATURES 24 
                                except AttributeError as e:  # geoCountry may not be found
                                    countryMatch = 'Unknown' 
                                # extract features related to webpage reputation in search engines
                                try:
                                    googleResultsReturned,googleResults = googleCustomSearchEngine(URL)
                                except googleapiclient.errors.HttpError as e: # error in google search engine
                                    print('Error 14a: ',e,'------',URL)
                                    continue
                                try :
                                    bingResultsReturned,bingResults = bingSearch(URL)
                                    URLMatch,engine = checkURLSearchEngineReputation(URL,googleResultsReturned,bingResultsReturned,googleResults,bingResults) # FEATURES 25
                                except json.decoder.JSONDecodeError as e: # error in bing search engine
                                    print('Error 14b: ',e,'------',URL)
                                    continue
                                domainMatch,FQDNMatch = checkDomainFQDNSearchEngineReputation(mainDomain,FQDN,googleResultsReturned,bingResultsReturned,googleResults,bingResults)
                                if FQDN != mainDomain :
                                    domainIPList = getIPAdresses(mainDomain) 
                                else:
                                    domainIPList = hostIPList
                                FQDNBlacklistedIP,domainBlacklistedIP,FQDNBlacklistIPCounts,domainBlacklistIPCounts = checkBlacklistedIPs(hostIPList,domainIPList,cur) 
                                 
                                # storing data of the extracted features into the database
                                URLID = SN
                                classLabel = 'Phishing'
                                try :
                                    recordDataset(cur,conn,URLID,URL,domainIdentityCounts,domainInCopyright,domainInCanonical,domainInAlternate,hyperlinkDomainMatched,linkRatio,formHandlerStatus,IPAddressUsed,domainEncoded,pathEncodedCharacters,redirectionCharacter,outPosition,noDotsFQDN,noDotsURLPath,standardPort,obfuscFQDN,obfuscPath,noSlashes,lengthFQDN,lengthPath,shortURLFound,freeDomainUsed,domainValidity,domainAge,FHDomainValidity,FHDomainAge,certificateType,countryMatch,URLMatch,domainMatch,FQDNMatch,FQDNBlacklistIPCounts,domainBlacklistIPCounts,classLabel)
                                except mysql.connector.errors.IntegrityError as e:  # skip duplicated records
                                    print(' Duplicate error 1! ---- ', URL)
                                    continue 
                                except mysql.connector.errors.DataError as e:  # skip duplicated records
                                    print(' Data too long for a column! ---- ', URL)
                                    continue
                                # record URL details in a separate database table
                                try:
                                    recordDetailedURL(cur,conn,URLID,URL,page_type,htmlEncodedJS,countText,countEmail,countTel,countDate,countMonth,countPass,noJSPrompts,lugha,voidLinks,samePageLinks,otherLinks)
                                except mysql.connector.errors.IntegrityError as e:  # skip duplicated records
                                    print(' Duplicate error 2! ---- ', URL)
                                    pass 
                                except mysql.connector.errors.DatabaseError as e:  # skip duplicated records
                                    print(' Database error! ---- ', URL)
                                    continue
                                
                                print(SN,'. ',URL)
                                SN = SN + 1
                        else:   # if a webpage does not have credentials prompting features or phrases, skip it go to the next one
                            print('No form or login phrases! --- ', URL)
                            continue   
    time_end = time.time()
    time_taken = (time_end - time_start) / 3600  
    time_taken = round(time_taken, 2)
    print ('----------- Recording of ', SN-1,' URLs completed after ', time_taken, ' hrs------------')        
    
                                                                      
urllib3.disable_warnings(InsecureRequestWarning)    # disable all warnings for disabling SSL verification
download_PhishTank_data()
conn,cur = openDBConnection2()
get_phishing_data(conn,cur)
closeDBConnection(conn,cur)   


